# Association Rule Mining with FP-Growth




### The data use is from UCI Machine Learning Repository.  The name of th data file is mushroom.data. The dataset has 23 features taken from species of gilled mushrooms and contains 8124 observations. Each of the features contains nominal values.

#####  Import Neccessary Libraries

In [2]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext

#### Create Spark Context for the communication between the client and the server

In [3]:
sc = SparkContext()

####  Import the data from a file and create an RDD because the algorithm expect a RDD

In [4]:
data = sc.textFile("mushroom.dat")

####  Parse the data line by line an create the needed strcuture for the algorithme. Each line is splitted by using as delimiter  the blank character (' '). And the map transformation create anothet RDD.

In [5]:
transactions = data.map(lambda line: line.strip().split(' '))

#### To limit the number of ItemSet the support is fixed to 0.9. For optimization resaon 10 partitions are use. The FRGrowth  algorithm is called via the train method of the FPGrowth object which is imporrted at the beginning of the file. The Train  method,as argument, expect an structured RDD, a minimum support and a number of partitions.

#####  With the trained data (Transactions data) the model is fitted and generated

In [6]:
model = FPGrowth.train(transactions, minSupport = 0.90, numPartitions = 10)

#### From the model the frquents ItemsSet are extracted by called the action Collect.

In [7]:
freqItemsets = model.freqItemsets().collect()

#### The number of itemSets is obtained by called the function len on the frquenset objec above. With the support of 90% the number of  FrequentItemSets is 9.

In [8]:
len(freqItemsets)

9

##### A Dictionnary is difined in order to tract the frequence of each itemSet in the DataSet (transaction Data). The Key is the ItemSDets and the values is the frequency.

In [9]:
freqItemsetsDict = {}

 #####  The freqence Items Set object obtained from the model is used to fill the above dictionnary

In [10]:
for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq

#####  To extract the the associated Associatiion Rule 60% of confidence level is used to minimize the number of rule. It is user difined value.

In [11]:
minConfidence = .6

#####  From the model and the confidence value the rule are extracted via the collect action. And the rule are sorted by confidencee via the sorted method.

In [ ]:
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), key=lambda x: x.confidence(), reverse=True)

#####  Get the numbeer rules by called the len method with argumeent as the rules object generated above. The number of rulesfor a confidence of 60% is 11.

In [13]:
len(rules)

11

From the rules object extracted above get for each rule the antecedent and the consequent and use  them with the dictionary freqItemsetsDict to 
calculate the support and the confidence. Use theses value for visualization (print).

In [14]:
for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

['34', '86'] => ['85'] (support: 0.97 confidence: 1.00)
['34'] => ['85'] (support: 0.97 confidence: 1.00)
['86'] => ['85'] (support: 0.98 confidence: 1.00)
['90'] => ['85'] (support: 0.92 confidence: 1.00)
['34'] => ['86'] (support: 0.97 confidence: 1.00)
['34', '85'] => ['86'] (support: 0.97 confidence: 1.00)
['86'] => ['34'] (support: 0.97 confidence: 1.00)
['86', '85'] => ['34'] (support: 0.97 confidence: 1.00)
['85'] => ['86'] (support: 0.97 confidence: 0.98)
['85'] => ['34'] (support: 0.97 confidence: 0.97)
['85'] => ['90'] (support: 0.97 confidence: 0.92)


Intrpretation for the rules aboves:

['34', '86'] => ['85'] (support: 0.97 confidence: 1.00) means 97% of the dataSet (transactions contains the item 34, 86 and 85) and each transaction that contains 34 and 86 conatins also 85 with certitude (100%)

['90'] => ['85'] (support: 0.92 confidence: 1.00)
eans 92% of the dataSet (transactions contains the item 90 and 85) and each transaction that contains 90 conatains also 85 with certitude (100%)

['85'] => ['86'] (support: 0.97 confidence: 0.98)
eans 97% of the dataSet (transactions contains the item 85 and 86) and each transaction that contains 85 conatins also 85 with a probability of 98%
    